In [1]:
import pandas as pd
import numpy as np
from ete3 import NCBITaxa
import boto3
import tempfile
import subprocess
import os
import io
import re
import time
import json
ncbi = NCBITaxa()

In [2]:
identity_qcov_threshold = 0.9
min_total_contig_len = 1000

In [3]:
## Load metadata
metadata = pd.read_csv("../../data/metadata/CMS001_CMS002_MergedAnnotations.csv", header=0)

In [ ]:
## Read counts data
all_read_counts = pd.read_csv("s3://czbiohub-mosquito/contig_quality_concat/contig_counts.csv", header=0)

In [ ]:
## Load data
contig_stats_lca = pd.read_csv("s3://czbiohub-mosquito/contig_quality_concat/contig_stats_lca.tsv", sep="\t", header=0)
contig_stats_lca = contig_stats_lca.assign(identity_qcov=(contig_stats_lca["identity"]/100*contig_stats_lca["align_length"]/contig_stats_lca["contig_length"]).apply(lambda x: min(x, 1)))
contig_stats_lca = contig_stats_lca[~(contig_stats_lca["hexapoda"] | contig_stats_lca["taxid"].isnull())]


In [ ]:
contig_stats_lca = pd.merge(pd.merge(contig_stats_lca, metadata, how="left", left_on="sample", right_on="NewIDseqName"), 
                            all_read_counts.groupby("sample").sum().reset_index().rename(columns={"counts":"total_nonhost"}),
                           how="left")

In [ ]:
contig_stats_lca["read_prop"] = contig_stats_lca["read_count"]/contig_stats_lca["total_nonhost"]



In [ ]:
def get_rows_taxid (df, taxid, taxid_colname="taxid", identity_qcov_cutoff=None):
    if (not isinstance(df, pd.DataFrame)):
        return (taxid in ncbi.get_lineage(df))
    outdf = df[df[taxid_colname].apply(get_rows_taxid, taxid=taxid)]
    if identity_qcov_cutoff is not None:
        outdf = outdf[outdf["identity_qcov"]>=identity_qcov_cutoff]
    return (outdf)

def filter_using_water_by_criterion (df, colname, split_by_taxid=True):
    if (len(df)==0):
        return (df)
    if split_by_taxid:
        water_counts = df[df["sample"].apply(lambda x: "ater" in x)].groupby(["taxid", "sample"])[colname].sum().reset_index().groupby("taxid").max()
        df_groupby = df.groupby(["taxid", "sample"])
        sample_sums = df_groupby[colname].sum().reset_index()
        for i, x in enumerate(water_counts[colname]):
            sample_sums["tokeep"] = sample_sums.loc[sample_sums["taxid"]==water_counts.index[i], colname] > x
            df = pd.merge(df, sample_sums.drop(columns=colname), how="left")
            df = df[df["tokeep"]!=False].drop(columns="tokeep")
    else:
        metrics = df.groupby("sample")[colname].sum().sort_values()
        selected_samples = metrics.index[([i for i, x in enumerate(metrics.index) if 'ater' in x][-1]+1):]
        df = df[df["sample"].isin(selected_samples)]
    return (df)
    
def use_water_filter (df, by_read_count=True, by_length=True, split_by_taxid=True):
    if (not df["sample"].apply(lambda x: "ater" in x).any()):
        return (df)
    if split_by_taxid:
        if (by_read_count):
            df = filter_using_water_by_criterion(df, "read_count", split_by_taxid)
        if (by_length):
            df = filter_using_water_by_criterion(df, "contig_length", split_by_taxid)
    else:
        if (by_read_count):
            filter_using_water_by_criterion(df, "read_count", split_by_taxid)
        if (by_length):
            filter_using_water_by_criterion(df, "contig_length", split_by_taxid)
    return (df)

def filter_by_criterion (df, colname, minthreshold, bysample=True):
    if bysample:
        sums = df.groupby(["sample", "taxid"])[colname].sum().reset_index()
        sums["tokeep"] = sums[colname] >= minthreshold
        df = pd.merge(df, sums.drop(columns=colname), how="left")
        df = df[df["tokeep"]!=False].drop(columns="tokeep")
    else:
        df = df[df[colname] >= minthreshold]
    return (df)


def check_if_in_any_taxid(taxid, taxid_list):
    if taxid in taxid_list:
        return (taxid)
    taxids = ncbi.get_lineage(taxid)
    check_in = [i for i, x in enumerate(taxids) if x in taxid_list]
    if (len(check_in)==0):
        return (np.nan)
    return (taxids[check_in[0]])



def clean_taxids(df, taxids, root_taxid, taxid_colname="taxid"):
    if isinstance(taxids[0], str):
        taxids = dict(zip([ncbi.get_name_translator([x])[x][0] for x in taxids], taxids))
    else:
        taxids = ncbi.get_taxid_translator(taxids)
    if isinstance(root_taxid, str):
        root_taxid_number = ncbi.get_name_translator([root_taxid])[root_taxid][0]
        root_taxid = {root_taxid_number:root_taxid}
    else:
        root_taxid = ncbi.get_taxid_translator([root_taxid])
    df[taxid_colname] = df[taxid_colname].apply(check_if_in_any_taxid, taxid_list=taxids)
    df[taxid_colname][df[taxid_colname].isnull()] = list(root_taxid.keys())[0]
    taxids.update(root_taxid)
    df["sci_name"] = df[taxid_colname].apply(lambda x: taxids[x])
    return (df)
    

def get_summary_table (df, colnames, metric):
    df = df.groupby(colnames)[metric].sum().reset_index()
    sort_order = colnames+[metric]
    sort_order.remove("sample")
    return (df.sort_values(by=sort_order, ascending=False))



    

In [ ]:
# Only keep hits that are almost identical to a known wolbachia sequence
wolbachia_taxid = 952
wolbachia_contigs = get_rows_taxid(contig_stats_lca, taxid=wolbachia_taxid, taxid_colname="taxid", identity_qcov_cutoff=identity_qcov_threshold)
# Filter contigs that were less abundant or shorter than contigs found in water belonging to the same taxid
wolbachia_contigs = use_water_filter(wolbachia_contigs, by_read_count=True, by_length=True)
# Filter taxids for which the total length of contigs in a sample was less than min_total_contig_len (default 1000 bp)
wolbachia_contigs = filter_by_criterion(wolbachia_contigs, "contig_length", minthreshold=min_total_contig_len, bysample=True)
# Return a list of species grouped by mosquito species, collection site, sample, and taxid, and sorted by total read count
wolbachia_contigs = get_summary_table(wolbachia_contigs, colnames=["ska_genus", "ska_species", "collected_by", "sample"], metric="read_prop")
wolbachia_contigs = wolbachia_contigs.assign(taxid=wolbachia_taxid)
wolbachia_contigs = wolbachia_contigs.assign(sci_name=ncbi.get_taxid_translator([wolbachia_taxid])[wolbachia_taxid])
    

In [ ]:
# Only keep hits that are almost identical to a known viral sequence
viral_contigs = get_rows_taxid(contig_stats_lca, taxid=10239, taxid_colname="taxid", identity_qcov_cutoff=identity_qcov_threshold)
# Filter contigs that were less abundant or shorter than contigs found in water belonging to the same taxid
viral_contigs = use_water_filter(viral_contigs, by_read_count=True, by_length=True, split_by_taxid=True)
# Filter taxids for which the total length of contigs in a sample was less than 500bp
viral_contigs = filter_by_criterion(viral_contigs, "contig_length", minthreshold=500, bysample=True)
# Convert taxids to those of interest
viral_groups = ["unclassified Riboviria", "Bunyavirales", "Orthomyxoviridae", "Mononegavirales", 
                "Nodaviridae", "Flavivirus", "Narnaviridae", "Totiviridae", "Iflaviridae", "dsRNA viruses"]
viral_contigs = clean_taxids(viral_contigs, taxids=viral_groups, root_taxid=10239)
# Return a list of species grouped by mosquito species, collection site, sample, and taxid, and sorted by total read count
viral_contigs = get_summary_table(viral_contigs, colnames=["ska_genus", "ska_species", "collected_by", "sample", "taxid", "sci_name"], metric="read_prop")

In [ ]:
# Only keep hits that are almost identical to a known metazoan sequence
metazoan_contigs = contig_stats_lca[(contig_stats_lca["taxon_group"]=="Metazoa") & (contig_stats_lca["identity_qcov"]>=identity_qcov_threshold)]
# Filter contigs that were less abundant or shorter than contigs found in water belonging to the same taxid
metazoan_contigs = use_water_filter(metazoan_contigs, by_read_count=True, by_length=True, split_by_taxid=True)
# Filter taxids for which the total length of contigs in a sample was less than min_total_contig_len (default 1000 bp)
metazoan_contigs = filter_by_criterion(metazoan_contigs, "contig_length", minthreshold=min_total_contig_len, bysample=True)
# Convert taxids to those of interest
metazoan_contigs = clean_taxids(metazoan_contigs, taxids=["Leporidae", "Muroidea", "Homo sapiens", "Carnivora", "Odocoileinae", "Bovidae", "Neognathae"], root_taxid="Metazoa")
# Return a list of species grouped by mosquito species, collection site, sample, and taxid, and sorted by total read count
metazoan_contigs = get_summary_table(metazoan_contigs, colnames=["ska_genus", "ska_species", "collected_by", "sample", "taxid", "sci_name"], metric="read_prop")

    

In [ ]:
# Only keep hits that are almost identical to a known eukaryotic sequence
eukaryotic_contigs = contig_stats_lca[(contig_stats_lca["taxon_group"]=="Eukaryota") & (contig_stats_lca["identity_qcov"]>=identity_qcov_threshold)]
# Filter contigs that were less abundant or shorter than contigs found in water belonging to the same taxid
eukaryotic_contigs = use_water_filter(eukaryotic_contigs, by_read_count=True, by_length=True, split_by_taxid=True)
# Filter taxids for which the total length of contigs in a sample was less than min_total_contig_len (default 1000 bp)
eukaryotic_contigs = filter_by_criterion(eukaryotic_contigs, "contig_length", minthreshold=min_total_contig_len, bysample=True)
# Convert taxids to those of interest
eukaryotic_groups = ["Fungi", "Trypanosomatidae", "Plasmodium", "Stramenopiles", "Viridiplantae", "Euglenozoa", "Alveolata"]
eukaryotic_contigs = clean_taxids(eukaryotic_contigs, taxids=eukaryotic_groups, root_taxid="Eukaryota")
# Return a list of species grouped by mosquito species, collection site, sample, and taxid, and sorted by total read count
eukaryotic_contigs = get_summary_table(eukaryotic_contigs, colnames=["ska_genus", "ska_species", "collected_by", "sample", "taxid", "sci_name"], metric="read_prop")


In [ ]:
all_contigs_df = pd.concat([wolbachia_contigs.assign(group="Wolbachia"), 
                            viral_contigs.assign(group="Virus"),
                            metazoan_contigs.assign(group="Metazoa"), 
                            eukaryotic_contigs.assign(group="Other Eukaryotes")])





In [ ]:
all_contigs_df.to_csv("../../figures/fig3/all_contigs_df.tsv", sep="\t", index=False)